<h2>Sentiment Analysis modified from LING 3832 Assignment</h2>

Generalised classifier using Naive Baye's (for now only using basic positive/negative classes)

* positive: `this is good`
* positive: `this could be great`
* negative: `this is not good`

In [1]:
# training data in dictionaries where  key = class
corpus = {}
corpus['pos'] = ["this is good", "this could be great"]
corpus['neg'] = ["this is not good"]

Store counts of each word in 'pos' and 'neg' using dictionaries again, function that calculates probability of a word belonging to a class, implement smoothing


In [2]:
counts = {}        # collect word counts in a dictionary of dictionaries
vocabulary = set() # store all the words used in a set

for c in corpus.keys():           # loop over sentiment classes
    counts[c] = {}                # create a counts dictionary for each class


# loop over classes, sentences, and words to add each word to the vocab set    
# for c in corpus.keys():
#     for text in corpus[c]:
#         for w in text.split():
#             vocabulary.add(w)
# more succint set comprehension:
vocabulary = {w for c in corpus.keys() for text in corpus[c] for w in text.split()}

# Set counts to zero in all classes
for word in vocabulary:
    for c in corpus.keys():
        counts[c][word] = 0
    
# create counts in the counts dictionaries
for c in corpus.keys():                     # each class
    for text in corpus[c]:                  # each sentence
        for w in text.split():              # each word/sentence
            counts[c][w] = counts[c][w] + 1 # +1 to count

In [3]:
vocabulary

{'be', 'could', 'good', 'great', 'is', 'not', 'this'}

In [4]:
counts

{'pos': {'not': 0,
  'great': 1,
  'good': 1,
  'is': 1,
  'be': 1,
  'could': 1,
  'this': 2},
 'neg': {'not': 1,
  'great': 0,
  'good': 1,
  'is': 1,
  'be': 0,
  'could': 0,
  'this': 1}}

In [5]:
# store sum total counts in each class in a totals dictionary
totals = {}
for c in corpus.keys():
    totals[c] = sum(counts[c].values())

In [6]:
totals

{'pos': 7, 'neg': 4}

function that returns the probability of given word in given class:  
smoothing applied with parameter of pseudocounts 'alpha'

In [7]:
def getprob(word, cl, countdict, totalsdict, vocabset, alpha):
    return (countdict[cl][word] + alpha)/(totalsdict[cl] + len(vocabset) * alpha)

In [8]:
# test
getprob('this', 'pos', counts, totals, vocabulary, 1.0)

0.21428571428571427

In [9]:
# test analysis
test = 'this is not great'

loop over each word in test sentence and multiply probabilities together from the pos and neg dictionaries, determine which class results in greater probablity

In [10]:
numdocuments = sum(len(corpus[c]) for c in corpus.keys())
prev = {}
for c in corpus.keys():
    prev[c] = len(corpus[c])/numdocuments

for c in corpus.keys():
    p = prev[c] # Set p to the prev
    for w in test.split():
        p = p * getprob(w, c, counts, totals, vocabulary, 1.0)
    print("The likelihood of the text: ", test, "  being in", c, "is", p)

The likelihood of the text:  this is not great   being in pos is 0.00020824656393169508
The likelihood of the text:  this is not great   being in neg is 0.00018213692143068556


same strategy using sum of log probabilities (above method returns small vals close to 0 for longer sentences)

In [11]:
import math
for c in corpus.keys():
    p = math.log(prev[c]) # Set p to the prior
    for w in test.split():
        p = p + math.log(getprob(w, c, counts, totals, vocabulary, 1.0))
    print("The log likelihood of the text: ", test, "  being in", c, "is", p)

The log likelihood of the text:  this is not great   being in pos is -8.4767877767812
The log likelihood of the text:  this is not great   being in neg is -8.610751838181756


conclusion:  
probability analysis returns that the sentence is slightly more likely to be positive, which is not correct but could be improved by weighting the positivity/negativity of each word in the vocab?  
also could be due to limited training data and when applied to larger corpora might perform more accurately?